### Building a Model To predict Emails Open Rates (OR%) , Based on the following Features 
- Time ( month of the year , quarter of the year , day of the week )
- Pirority of email , main or reminder 
- Category ( interval , final )
- Number of emails sent in each subject
- Constructed Features
 - Length of the subject 
 - Personalization of Email
 - Having emoji or not
 - Casual tone or formal 
 - Urgency at the subject 
 - Metion of price or discount in the subject 
 - Imperative tone in the subject 
 - Mention of product in the subject

In [ ]:
#importing libraries 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [ ]:
#r reading the data as a dataframe
df = pd.read_csv("model_data.csv")

In [ ]:
df.info()

In [ ]:
columns_to_drop = [ "translation", "CTR clean %","Date","Month"]  
df = df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Convert to datetime
df['Sent date'] = pd.to_datetime(df['Sent date'], errors='coerce')

# Create a Month column (numeric)
df['Month'] = df['Sent date'].dt.month


In [ ]:

import re

def count_word_letters(text):
    text = str(text)
    
    # 1️⃣ Remove emoji codes like &#x1f4b0; or &#128512;
    text = re.sub(r'&#x?[0-9A-Fa-f]+;?', '', text)
    
    # 2️⃣ Remove template/formatted patterns {% ... %} and {{ ... }}
    text = re.sub(r'{%.*?%}', '', text)
    text = re.sub(r'{{.*?}}', '', text)
    
    # 3️⃣ Remove HTML entities like &amp;, &lt;, etc.
    text = re.sub(r'&[a-zA-Z]+;', '', text)
    
    # 4️⃣ Count only alphabetic letters (a-z, A-Z)
    return len(re.findall(r'[A-Za-z]', text))

# Apply to your DataFrame
df['letter_count'] = df['Subject'].apply(count_word_letters)



now i will drop subject


In [ ]:
df = df.drop(columns='Subject', errors='ignore')

Next , I will confirm that the values in "OR clean %" is a valid percentage 

In [ ]:
# Check if all values are between 0 and 1
valid_percentage = df['OR clean %'].between(0, 1).all()

if valid_percentage:
    print("All values in 'OR clean %' are valid percentages (0 to 1).")
else:
    # Show which rows are invalid
    invalid_rows = df[~df['OR clean %'].between(0, 1)]
    print("Some values are outside 0 to 1:")
    display(invalid_rows)


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def get_column_outliers(df, columns=['Sendings', 'OR clean %'], threshold=3):
    """
    Detect outliers in each specified numeric column using Z-score.
    
    Parameters:
    - df: pandas DataFrame
    - columns: list of column names to check
    - threshold: Z-score cutoff (default 3)
    
    Returns:
    - Dictionary: column name -> DataFrame of outlier rows
    """
    outlier_dict = {}
    
    for col in columns:
        if col in df.columns:
            col_data = df[col]
            z_scores = np.abs(stats.zscore(col_data.dropna()))
            col_outliers = df.loc[col_data.dropna().index[z_scores > threshold]]
            outlier_dict[col] = col_outliers
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")
            outlier_dict[col] = pd.DataFrame()  # empty DataFrame if column not found
    
    return outlier_dict

# Example usage
outliers = get_column_outliers(df, columns=['Sendings', 'OR clean %'], threshold=3)

print("Outliers in 'Sendings':")
display(outliers['Sendings'])

print("Outliers in 'OR clean %':")
display(outliers['OR clean %'])


In [ ]:


# Example: ensure units are consistent first
# For demonstration, assume Sendings are already in correct units
# If not, apply a conversion here, e.g.,
# df['Sendings'] = df['Sendings'] * 1000  # convert thousands to units

# Set Winsorization thresholds (percentiles)
lower_pct = 0.01  # 1st percentile
upper_pct = 0.99  # 99th percentile

# List of columns to Winsorize
columns_to_winsorize = ['Sendings', 'OR clean %']

for col in columns_to_winsorize:
    lower = df[col].quantile(lower_pct)
    upper = df[col].quantile(upper_pct)
    df[col] = np.clip(df[col], lower, upper)  # Cap values at thresholds
    print(f"{col}: capped below {lower:.3f}, capped above {upper:.3f}")

# Now df has Winsorized columns ready for modeling


- Confirming that Data is valid for regression models 
 - Cheking correlation 
 - Ckeking multicoliniarity

In [ ]:
# code to check correlation 
target = "OR clean %"  

numeric_df = df.select_dtypes(include=['int64', 'float64'])
corr_with_target = numeric_df.corr()[target].sort_values(ascending=False)

print("\n✅ Correlation with target:")
print(corr_with_target)

In [ ]:
# code to check multicolinearity 
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

X = numeric_df.drop(columns=[target])  
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\n✅ VIF to detect multicollinearity:")
print(vif_data)

The previous result shows that there is no multicolinearity effect 


### Building bionomial GLM  model 

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# 1. Imports
# ================================
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
# Keep only valid OR clean % values
# df = df[(df['OR clean %'] >= 0) & (df['OR clean %'] <= 1)].copy()

# Convert 'Sent date' to datetime
# df['Sent date'] = pd.to_datetime(df['Sent date'], errors='coerce')

In [ ]:
df['opens'] = (df['OR clean %'] * df['Sendings']).round().astype(int)
df['fails'] = (df['Sendings'] - df['opens']).round().astype(int)

In [ ]:
categorical_cols = ['DD', 'Category name', 'Main vs REM', 'Month']

# Create dummies
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Select predictor columns (exclude target and weights)
predictor_cols = [c for c in df.columns if c not in ['OR clean %', 'Sendings', 'opens', 'fails']]

# Ensure numeric float
df[predictor_cols] = df[predictor_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)


In [ ]:
X = df[predictor_cols].astype(float)
y = df[['opens','fails']].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test, has_constant='add')

In [ ]:
# Assuming X_train is the preprocessed DataFrame used in GLM training
import pandas as pd

# 1️⃣ Get all categorical columns (dummy columns)
dummy_cols = [col for col in X_train.columns if '_' in col]

# 2️⃣ Map each original categorical column to its categories
categorical_columns = {}
for col in dummy_cols:
    # The dummy column name format is usually: 'OriginalColumn_Category'
    original_col, category = col.split('_', 1)
    if original_col not in categorical_columns:
        categorical_columns[original_col] = []
    categorical_columns[original_col].append(category)

print("Categorical columns and their categories:")
for col, cats in categorical_columns.items():
    print(f"{col}: {cats}")


In [ ]:
X_train.head()

In [ ]:
glm_binom = sm.GLM(y_train, X_train, family=sm.families.Binomial())
model_result = glm_binom.fit()
print(model_result.summary())

In [ ]:
y_pred_or = model_result.predict(X_test)

In [ ]:
y_test_rate = y_test[:,0] / (y_test[:,0] + y_test[:,1])

In [ ]:
mae = mean_absolute_error(y_test_rate, y_pred_or)
print(f"MAE on test set: {mae:.4f}")

In [ ]:


df_test_eval = X_test.copy()
df_test_eval['y_pred'] = y_pred_or
df_test_eval['y_true_rate'] = y_test_rate

top_10pct = df_test_eval.sort_values('y_pred', ascending=False).head(int(len(df_test_eval)*0.1))
top_decile_mean = top_10pct['y_true_rate'].mean()
overall_mean = y_test_rate.mean()
lift = top_decile_mean / overall_mean

# -------------------------------
# 6. Mean predicted vs mean actual
# -------------------------------
mean_pred = np.mean(y_pred_or)
mean_actual = np.mean(y_test_rate)

print(f"Top-decile lift: {lift:.2f}")
print(f"Mean predicted OR clean %: {mean_pred:.4f}")
print(f"Mean actual OR clean %:    {mean_actual:.4f}")
# -------------------------------



In [ ]:
# ================================


# ================================
# 2. Prepare Data
# ================================


# Compute successes/failures for Binomial GLM as integers




# ================================
# 3. Preprocessing: categorical → dummies
# ================================

# ================================
# 4. Train/Test Split (random)
# ================================




# Add intercept


# ================================
# 5. Fit Binomial GLM
# ================================


# ================================
# 6. Predictions on test set
# ================================


# True rate for evaluation


# ================================
# 7. Evaluation Metrics
# ================================
# MAE


# Calibration curve
plt.scatter(y_pred_or, y_test_rate, alpha=0.5)
plt.plot([0,1],[0,1], linestyle='--', color='gray')
plt.xlabel("Predicted OR clean %")
plt.ylabel("Actual OR clean %")
plt.title("Calibration Scatter")
plt.show()

# Top-decile lift
df_test_eval = X_test.copy()
df_test_eval['y_pred'] = y_pred_or
df_test_eval['y_true_rate'] = y_test_rate

top_10pct = df_test_eval.sort_values('y_pred', ascending=False).head(int(len(df_test_eval)*0.1))
top_decile_mean = top_10pct['y_true_rate'].mean()
overall_mean = y_test_rate.mean()
lift = top_decile_mean / overall_mean
print(f"Top-decile lift: {lift:.2f}")


In [ ]:
def predict_or_and_opens(model, emails_list, predictor_cols, categorical_cols=['DD', 'Category name', 'Main vs REM', 'Month']):
    """
    Predict OR clean % and estimated opens/fails for one or multiple emails.

    Parameters:
    -----------
    model : statsmodels.GLMResults
        Trained Binomial GLM model.
    emails_list : list of dicts
        Each dict contains features for one email. Must include 'Sendings'.
    predictor_cols : list
        Numeric + dummy predictor columns used in training (exclude 'Sendings').
    categorical_cols : list
        Categorical columns to convert to dummies.

    Returns:
    --------
    result_df : pandas.DataFrame
        DataFrame containing predicted OR clean %, predicted opens, predicted fails.
    """
    import pandas as pd
    import statsmodels.api as sm
    import numpy as np

    # Convert list of dicts to DataFrame
    df_new = pd.DataFrame(emails_list)

    if 'Sendings' not in df_new.columns:
        raise ValueError("Each email must include 'Sendings' for trial counts.")

    # Convert 'Sent date' to numeric if present
    if 'Sent date' in df_new.columns:
        df_new['Sent_date_ordinal'] = pd.to_datetime(df_new['Sent date'], errors='coerce').map(lambda x: x.toordinal())

    # Create dummies
    df_new = pd.get_dummies(df_new, columns=categorical_cols, drop_first=True)

    # Add missing columns as zeros
    for col in predictor_cols:
        if col not in df_new.columns:
            df_new[col] = 0

    # Keep only predictor columns in the correct order
    X_new = df_new[predictor_cols].astype(float)
    X_new = sm.add_constant(X_new, has_constant='add')

    # Predict OR clean %
    y_pred_or = model.predict(X_new)

    # Calculate predicted opens/fails
    pred_opens = y_pred_or * df_new['Sendings']
    pred_fails = df_new['Sendings'] - pred_opens

    # Combine into result DataFrame
    result_df = df_new.copy()
    result_df['Predicted_OR'] = y_pred_or
    result_df['Predicted_Opens'] = pred_opens
    result_df['Predicted_Fails'] = pred_fails

    return result_df[['Predicted_OR', 'Predicted_Opens', 'Predicted_Fails']]


In [ ]:
emails = [
    {
        'Sendings': 1500,
        'length of subject': 45,
        'Personalization': 1,
        'Emoji': 0,
        'Urgency': 1,
        'Tone': 0,
        'Price or Discount': 1,
        'imperative ': 0,
        'product': 1,
        'Month': 'Month_11',
        'DD': "DD_Thursday",
        'Category name': 'Promo',
        'Main vs REM': 'Main'
    },
    {
        'Sendings': 800,
        'length of subject': 30,
        'Personalization': 0,
        'Emoji': 1,
        'Urgency': 0,
        'Tone': 1,
        'Price or Discount': 0,
        'imperative ': 1,
        'product': 0,
        'Month': 'Month_02',
        'DD': 'DD_Tuesday',
        'Category name': 'Update',
        'Main vs REM': 'REM'
    }
]

results = predict_or_and_opens(model_result, emails, predictor_cols)
print(results)
